### VERİ SETİ: 
- Farklı markaya sahaip hoparlörlerin bilgileri yer alıyor.

### KOLONLAR:
1. review ID: Her benzersiz yorum için id
2. title: Kullanıcı görüşünün özeti
3. author: İncelemeyi yapan kişi
4. rating: Kullanıcının verdiği puan(5 üzerinden)
5. content: Kullanıcı yorumu
6. timestamp: Yorumun ne zaman ve nereden yapıldığı
7. Is_verified: Yorumu yapan kullanıcın ürünü alıp almadığı
8. helpful_count: Yapılan yorumun kaç kişi tarafından beğenildiği
9. product_attributes: Hoparlör özellikleri
10. Product_name: Ürün adı
11. price: Ürün fiyatı
12. company: Şirket adı

In [1]:
import pandas as pd
import re
import string
import nltk

# nltk.download('stopwords', force=True)
# nltk.download('wordnet', force=True)
# nltk.download('omw-1.4', force=True)
# nltk.download('punkt', force=True)
# nltk.download('punkt_tab')

from nltk.corpus import stopwords # Etkisisiz/Anlamsız(Stopwords) kelimelerin kaldırılması için
from nltk.stem import WordNetLemmatizer # Lemmatization: Kelimelerin kökününe dönüştürme için 

from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDF hesaplaması için
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix

import pickle

In [2]:
df = pd.read_csv("urun_analizi.csv")
df.head(1)

,Unnamed: 0,review_id,title,author,rating,content,timestamp,is_verified,helpful_count,product_attributes,Product_name,price(in $),company
0,0,RE2FG9LL12TYG,5.0 out of 5 stars Excellent Design and Sound ...,luis miguel,5,"It has a beautiful design, great sound quality...","Reviewed in the United States September 2, 2024",True,0,Color: Black & BrassStyle: Emberton II,Marshall Emberton II,115,Marshall


In [3]:
# Kolonları silme
df.drop(["Unnamed: 0", "review_id", "title", "author", "timestamp", "is_verified", "helpful_count"], axis=1, inplace=True)

In [4]:
# Eksik veri Kontrolü
df.isnull().sum()

rating                0
content               2
product_attributes    0
Product_name          0
price(in $)           0
company               0
dtype: int64

In [5]:
df.dropna(inplace=True)

### VERİ TEMİZLEME

In [6]:
# "product_attributes" kolonunda sadece renk kısmını ele alma
def renk(text):
    ayrim = re.search(r'Color:\s*([A-Za-z\s&]+)', text)
    if ayrim:
        return ayrim.group(1).strip()
    else:
        # Color: yoksa orijinal metni döndür (örneğin "black", "grey" gibi)
        return text.strip()
colors = [renk(item) for item in df["product_attributes"]]
df["product_attributes"] = colors

In [7]:
# Tüm harflerin küçük harfe dönüştürülmesi
df[df.select_dtypes(include=["object"]).columns] = df.select_dtypes(include=["object"]).apply(lambda x:x.str.lower())

# Noktalama işaretlerinin kaldırılması
df[df.select_dtypes(include=["object"]).columns] = df.select_dtypes(include=["object"]).apply(lambda x:x.str.translate(str.maketrans('', '', string.punctuation)))

# Metin içerisindeki sayısal değerlerden kurtulma
df[df.select_dtypes(include=["object"]).columns] =  df.select_dtypes(include=["object"]).apply(lambda x:x.str.replace(r'\d+', '', regex=True))

# Etkisisiz/Anlamsız(Stopwords) kelimelerin kaldırılması
stop_words = set(stopwords.words('english'))
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].apply(lambda text: ' '.join([word for word in text.split() if word.lower() not in stop_words]) if isinstance(text, str) else text)
    
# Lemmatization: Kelimelerin kökününe dönüştürme
lemmatizer = WordNetLemmatizer()

for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].apply(lambda text: ' '.join([lemmatizer.lemmatize(word) for word in text.split()]))

In [10]:
# Ordinal Encoding
rating_siralama = [[0.0, 1.0, 2.0 ,3.0 ,4.0 ,5.0]]

ordinal_coding = OrdinalEncoder(categories=rating_siralama)
df["rating"] = ordinal_coding.fit_transform(df[["rating"]])

### TF-IDF ve  OneHotEncoder, StandardScaler İşlemleri

In [11]:
tfidf = TfidfVectorizer(max_features=1000)

preprocessor = ColumnTransformer(
    transformers=[
        ('tfidf', tfidf, 'content'), 
        ('num', StandardScaler(), ['rating', 'price(in $)']), 
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['product_attributes', 'company'])
    ])

X = preprocessor.fit_transform(df) # bağımsız değişkenler

In [12]:
y = df["Product_name"] # bağımlı değişken

In [13]:
# Eğitim ve test kümesi ayrımı
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Model kurma ve eğitme
model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [15]:
# Tahmin etme
y_pred = model.predict(x_test)

In [16]:
# Oluşturulan TF-IDF vekterünü ve modeli kaydetme(LogisticRegression)
with open('tfidf_vectorizer_3.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)

with open('product_prediction_model_3.pkl', 'wb') as f:
    pickle.dump(model, f)

### Modellin Kullanılması

In [5]:
with open('tfidf_vectorizer_3.pkl', 'rb') as f:
    preprocessor = pickle.load(f)

with open('product_prediction_model_3.pkl', 'rb') as f:
    model = pickle.load(f)

user_content = input("Nasıl bir ürün almak istediğinizi açıkla açıklar mısınız? (content): ")
user_rating = float(input("Almak istediğiniz ürün kaç puan almış olsun? (rating): "))
user_attributes = input("Ürün rengini giriniz (örnek: black, white): ")
user_company = input("Hangi marka olsun istersiniz? (örnek: marshall): ")


user_price_range = input("Almak istediğiniz ürün için fiyat aralığını giriniz (örnek: 150-200): ")
user_price_min, user_price_max = map(float, user_price_range.split('-'))  # Fiyat aralığını ayır
user_price_avg = (user_price_min + user_price_max) / 2


user_content_lower = user_content.lower()
user_attributes_lower = user_attributes.lower()
user_company_lower = user_company.lower()



new_data = {
    'content': [user_content], 
    'rating': [user_rating], 
    'price(in $)': [user_price_avg], 
    'product_attributes': [user_attributes],  
    'company': [user_company] 
}


new_data_df = pd.DataFrame(new_data)

user_input_transformed = preprocessor.transform(new_data_df)

predicted_product = model.predict(user_input_transformed)

print("Tahmin edilen ürün:", predicted_product[0])

Nasıl bir ürün almak istediğinizi açıkla açıklar mısınız? (content): Kulağı rahatsız etmeyen bir ürün istiyorum.
Almak istediğiniz ürün kaç puan almış olsun? (rating): 4
Ürün rengini giriniz (örnek: black, white): Siyah
Hangi marka olsun istersiniz? (örnek: marshall): sony
Almak istediğiniz ürün için fiyat aralığını giriniz (örnek: 150-200): 150-200
Tahmin edilen ürün: bang olufsen beosound
